<a href="https://colab.research.google.com/github/ktzh/NLPhomeworks/blob/main/Lecture04_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задачи к Лекции 4

__Исходные данные__

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]


df = pd.read_csv("/content/mlbootcamp5_train.csv",
                 sep=";",
                 index_col="id")
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,True,False,False,True,False,False,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,False,False,True,True,False,False,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,False,False,True,True,False,False,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,True,False,False,True,False,False,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,True,False,False,True,False,False,0


## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедент.

In [3]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from collections import Counter
from sklearn.model_selection import KFold

class KNNClassifier:
    def __init__(self, k=5, metric='euclidean', weights='uniform'):
        self.k = k
        self.metric = metric
        self.weights = weights

    def fit(self, X, y):
        self.X_train = np.array(X.values if isinstance(X, pd.DataFrame) else X, dtype=np.float64)
        self.y_train = np.array(y.values if isinstance(y, pd.Series) else y)

    def predict(self, X):
        X_test = np.array(X.values if isinstance(X, pd.DataFrame) else X, dtype=np.float64)
        predictions = [self._predict(x) for x in X_test]
        return np.array(predictions)

    def _predict(self, x):
        # 1. Вычисляем расстояния
        if self.metric == 'euclidean':
            dists = np.linalg.norm(self.X_train - x, axis=1)
        elif self.metric == 'manhattan':
            dists = np.sum(np.abs(self.X_train - x), axis=1)
        else:
            dists = distance.cdist([x], self.X_train, self.metric)[0]

        # 2. Сортируем и выбираем k ближайших
        k_indices = np.argsort(dists)[:self.k]
        k_nearest_labels = self.y_train[k_indices]

        # 3. Взвешенное голосование
        if self.weights == 'uniform':
            most_common = Counter(k_nearest_labels).most_common(1)
            return most_common[0][0]
        else:
            weights = 1 / (dists[k_indices] + 1e-10)  # Чтобы избежать деления на 0
            weighted_votes = {}
            for label, weight in zip(k_nearest_labels, weights):
                weighted_votes[label] = weighted_votes.get(label, 0) + weight
            return max(weighted_votes.items(), key=lambda x: x[1])[0]

def cross_val_knn(X, y, k_values=range(1, 21), cv=5):
    kf = KFold(n_splits=cv)
    best_k = None
    best_score = -1
    scores = {}

    for k in k_values:
        knn = KNNClassifier(k=k)
        fold_scores = []

        for train_idx, val_idx in kf.split(X):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            knn.fit(X_train, y_train)
            preds = knn.predict(X_val)
            accuracy = np.mean(preds == y_val)
            fold_scores.append(accuracy)

        avg_score = np.mean(fold_scores)
        scores[k] = avg_score
        print(f"k={k}: Accuracy = {avg_score:.3f}")

        if avg_score > best_score:
            best_score = avg_score
            best_k = k

    print(f"\nBest k: {best_k} with accuracy {best_score:.3f}")
    return best_k, scores

features = ['gender_bin', 'chol_1', 'chol_2', 'chol_3', 'gluc_1', 'gluc_2', 'gluc_3']
X = df[features].astype(float)
y = df['cardio']

best_k, scores = cross_val_knn(X, y, k_values=range(1, 11))

final_knn = KNNClassifier(k=best_k)
final_knn.fit(X, y)

# Пример предсказания
sample = X.iloc[0:1]  # Первый образец
pred = final_knn.predict(sample)
print(f"\nПредсказание для образца:\n{sample}\nPrediction: {pred[0]}")

k=1: Accuracy = 0.533
k=2: Accuracy = 0.533
k=3: Accuracy = 0.559
k=4: Accuracy = 0.553
k=5: Accuracy = 0.561
k=6: Accuracy = 0.559
k=7: Accuracy = 0.574
k=8: Accuracy = 0.555
k=9: Accuracy = 0.574
k=10: Accuracy = 0.571
k=11: Accuracy = 0.564
k=12: Accuracy = 0.558
k=13: Accuracy = 0.565
k=14: Accuracy = 0.558
k=15: Accuracy = 0.565
k=16: Accuracy = 0.565
k=17: Accuracy = 0.562
k=18: Accuracy = 0.566
k=19: Accuracy = 0.565
k=20: Accuracy = 0.553

Best k: 7 with accuracy 0.574

Предсказание для образца:
    gender_bin  chol_1  chol_2  chol_3  gluc_1  gluc_2  gluc_3
id                                                            
0          1.0     1.0     0.0     0.0     1.0     0.0     0.0
Prediction: 0


**Комментарии:** Ваши комментарии здесь.

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

features = ['age', 'weight', 'height', 'ap_lo', 'ap_hi']
X = df[features]
y = df['cardio']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models = {
    "kNN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Decision Tree": DecisionTreeClassifier(random_state=42)
}

results = []

for model_name, model in models.items():
    print(f"Обучение модели: {model_name}")

    if model_name == "kNN":
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    ll = log_loss(y_test, y_proba)

    results.append({
        "Model": model_name,
        "Accuracy": acc,
        "F1-score": f1,
        "ROC AUC": auc,
        "Log Loss": ll
    })

results_df = pd.DataFrame(results).set_index("Model")
print("\nРезультаты:")
print(results_df.round(4))

Обучение модели: kNN
Обучение модели: Naive Bayes
Обучение модели: Decision Tree

Результаты:
               Accuracy  F1-score  ROC AUC  Log Loss
Model                                               
kNN              0.6585    0.6504   0.7021    2.1272
Naive Bayes      0.5412    0.2136   0.6675    0.7851
Decision Tree    0.6176    0.6176   0.6177   13.7693


**Комментарии:** Ваши комментарии здесь.